# Calls

In [57]:
from pymongo import MongoClient

import time

MONGO_HOST="localhost"
MONGO_USER="root"
MONGO_PASSWORD="password"
MONGO_PORT=27017

address = 'mongodb://{user}:{password}@{host}:{port}'.format(
    user= MONGO_USER,
    password= MONGO_PASSWORD,
    host= MONGO_HOST,
    port= MONGO_PORT,
)

client = MongoClient(address)
database=client["observertc-reports-20"]

# the collection we want to query
reportsDatabase = database.reports

# print('connected to %s' % address)

cursor = reportsDatabase.find( { "type": "CALL_EVENT", "payload.name": { "$in": ["CLIENT_JOINED", "CLIENT_LEFT"] } } )

rooms = {}
for record in cursor:
    callEvent = record["payload"]

    roomId = callEvent["roomId"]
    callId = callEvent["callId"]
    userId = callEvent['userId']
    clientId = callEvent['clientId']
    timestamp = callEvent['timestamp']

    calls = rooms.get(roomId, {})
    clients = calls.get(callId, {})
    client = clients.get(clientId, {
        "clientId": clientId,
        "userId": userId
    })
    
    if callEvent["name"] == "CLIENT_JOINED":
        client.update({ "joined": timestamp })
    else:
        client.update({ "left": timestamp })
    
    clients[clientId] = client
    calls[callId] = clients
    rooms[roomId] = calls

for room in rooms.items():
    roomId, calls = room
    print('\nRoom: %s\n--------' % roomId)
    for call in calls.items():
        callId, clients = call
        print('\nCall %s' % callId)
        for client in clients.values():
            userId, clientId, joined = client["userId"], client["clientId"], client["joined"]
            left = client.get("left", int(time.time() * 1000))
            duration = (left - joined) / (60 * 1000)
            print('\n\tUserId: %s \n\tClientId: %s\n\tJoined: %d\n\tLeft: %d\n\tDuration: %d min' % 
                  (userId, clientId, joined, left, duration))


Room: Maputo
--------

Call 5a7d1dbc-9c4b-4d8d-9f1d-db16ecb46243

	UserId: Seamus Finnigan 
	ClientId: dd649fa3-8ae5-443f-bf54-0d678e3bec39
	Joined: 1652693994682
	Left: 1652694207800
	Duration: 3 min

	UserId: Trelawney 
	ClientId: b08c1107-779e-4e22-ad90-19a86c5323f3
	Joined: 1652693990493
	Left: 1652694207804
	Duration: 3 min


# Max concurrent streams through SFU over time

In [45]:
# Match all tracks connected to an sfu
stage_match = {
    "$match": {
        "$or": [{ "type": "INBOUND_VIDEO_TRACK" }, { "type": "INBOUND_AUDIO_TRACK" },
                { "type": "OUTBOUND_VIDEO_TRACK" }, { "type": "OUTBOUND_AUDIO_TRACK" }],
        "$or": [{"payload.sfuStreamId": { "$exists" : True }}, 
                {"payload.sfuSinkId": { "$exists" : True }, "payload.sfuStreamId": { "$exists" : True }}]
   }
}
# group by clients
stage_group = {
   "$group": {
         "_id": {
             "clientId": "$payload.clientId",
             "sfuSinkId": "$payload.sfuSinkId",
             "sfuStreamId": "$payload.sfuStreamId"
          },
           "reportsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of concurrent sfu streams", len(list(cursor)))
# num = 0
# for report in cursor:
#     print(report)
#     num = num + 1
# print(num)

{'_id': {'clientId': 'b08c1107-779e-4e22-ad90-19a86c5323f3', 'sfuStreamId': 'f589f569-911b-4e68-8429-0fa3e7df34e0'}, 'reportsNum': 19}
{'_id': {'clientId': 'dd649fa3-8ae5-443f-bf54-0d678e3bec39', 'sfuSinkId': '01034106-862b-475e-a8de-7a38ebdf787c', 'sfuStreamId': 'f589f569-911b-4e68-8429-0fa3e7df34e0'}, 'reportsNum': 18}
{'_id': {'clientId': 'dd649fa3-8ae5-443f-bf54-0d678e3bec39', 'sfuStreamId': 'cd6fc1b7-fcda-4578-9720-6f69fd7dd033'}, 'reportsNum': 19}
{'_id': {'clientId': 'dd649fa3-8ae5-443f-bf54-0d678e3bec39', 'sfuSinkId': '127180d1-6a96-4d84-8dfc-c7bfcab3e733', 'sfuStreamId': '24ee86df-1f24-4567-92ee-5df88c0983e2'}, 'reportsNum': 18}
{'_id': {'clientId': 'b08c1107-779e-4e22-ad90-19a86c5323f3', 'sfuSinkId': 'e2d466f9-ec36-4315-a2ee-e279a2f7e963', 'sfuStreamId': 'cd6fc1b7-fcda-4578-9720-6f69fd7dd033'}, 'reportsNum': 18}
{'_id': {'clientId': 'b08c1107-779e-4e22-ad90-19a86c5323f3', 'sfuStreamId': '24ee86df-1f24-4567-92ee-5df88c0983e2'}, 'reportsNum': 19}
{'_id': {'clientId': 'b08c1107-

In [47]:
# Match all RTP pad joined to the SFU
stage_match = {
    "$match": {
        "$or": [{ "type": "SFU_INBOUND_RTP_PAD" }, { "type": "SFU_OUTBOUND_RTP_PAD" }]
   }
}
# group by clients
stage_group = {
   "$group": {
         "_id": {
             "sfuId": "$payload.sfuId",
             "sfuSinkId": "$payload.sfuSinkId",
             "sfuStreamId": "$payload.sfuStreamId"
         },
         "reportsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of concurrent sfu streams", len(list(cursor)))
num = 0
for report in cursor:
    print(report)
    num = num + 1
print(num)

number of concurrent sfu streams 8
0


# Number of calls over time

In [60]:
# Match all calls started
stage_match = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CALL_STARTED",
   }
}


# the pipeline
pipeline = [
   stage_match, 
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of calls", len(list(cursor)))
num = 0

number of calls 1


# % conferences with 2 participants over time

In [87]:
# Match all tracks connected to an sfu
stage_match_joined_clients = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CLIENT_JOINED",
   }
}
# group by callId
stage_group_by_callId = {
   "$group": {
         "_id": "$payload.callId",
         "clientsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match_joined_clients, 
   stage_group_by_callId,
]

cursor = reportsDatabase.aggregate(pipeline)
total_conferences = 0
p2p_conferences = 0
for report in cursor:
    total_conferences += 1
    p2p_conferences += 1 if report["clientsNum"] == 2 else 0
print("number of concurrent sfu streams", len(list(cursor)))

% conferences with 2 participants over time 100.0


# Average conference size

In [88]:
# Match all tracks connected to an sfu
stage_match_joined_clients = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CLIENT_JOINED",
   }
}
# group by callId
stage_group_by_callId = {
   "$group": {
         "_id": "$payload.callId",
         "clientsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match_joined_clients, 
   stage_group_by_callId,
]

cursor = reportsDatabase.aggregate(pipeline)
total_conferences = 0
nr_of_clients = 0
for report in cursor:
    total_conferences += 1
    nr_of_clients += report["clientsNum"]

print("Average conference size", nr_of_clients / total_conferences)

Average conference size 2.0


# Number of unique users (unique cookie ids for a given time period)

In [89]:
# Match all tracks connected to an sfu
stage_match_joined_clients = {
    "$match": {
        "type": "CLIENT_EXTENSION_DATA",
        "payload.type": "BROWSER_FINGERPRINT",
   }
}
# group by callId
stage_group_by_fingerpring = {
   "$group": {
         "_id": "$payload.payload",
   }
}

# the pipeline
pipeline = [
   stage_match_joined_clients, 
   stage_group_by_callId,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of unique users", len(list(cursor)))

number of concurrent sfu streams 0


# % of user joins with media issues over time

In [93]:
cursor = reportsDatabase.find({
    "type": "CALL_EVENT",
    "payload.name": "CLIENT_JOINED"
})
number_of_clients = len(list(cursor))

cursor = reportsDatabase.find({
    "type": "CALL_META_DATA",
    "payload.type": "USER_MEDIA_ERROR"
})

number_of_user_media_errors = len(list(cursor))

print("% of user joins with media issues over time", number_of_user_media_errors / number_of_clients)

% of user joins with media issues over time 1.0


# % of user joins (or join attempts) with connectivity issues over time

In [94]:
cursor = reportsDatabase.find({
    "type": "CALL_EVENT",
    "payload.name": "CLIENT_JOINED"
})
number_of_clients = len(list(cursor))

cursor = reportsDatabase.find({
    "type": "CLIENT_EXTENSION_DATA",
    "payload.type": "ICE_DISCONNECTED"
})

number_of_connectivity_issue = len(list(cursor))

print("% of user joins (or join attempts) with connectivity issues over time", number_of_connectivity_issue / number_of_clients)



% of user joins (or join attempts) with connectivity issues over time 0.0
